In [13]:
import numpy as np
import pandas as pd
import pickle
import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.metrics import classification_report

Load dataset and get binary and multiclass y columns.

In [14]:
binary = pd.read_csv("binary.csv")
binary.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,6,119360531,136,213,41528.0,155400.0,2955.0,0.0,305.352941,610.445587,...,1.522822e+06,2403023.0,249428.0,58229360.5,565451.372605,58629195.0,57829526.0,0.0,0,BENIGN
1,6,118838133,40,43,4216.0,16444.0,696.0,0.0,105.400000,188.610683,...,8.040554e+05,1152339.0,15233.0,58827757.0,223646.561181,58985899.0,58669615.0,0.0,0,BENIGN
2,6,108,1,2,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,BENIGN
3,17,21925,2,2,62.0,94.0,31.0,31.0,31.000000,0.000000,...,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,BENIGN
4,6,141,1,2,6.0,12.0,6.0,6.0,6.000000,0.000000,...,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,BENIGN


In [15]:
y = binary[" Label"].copy()
X = binary.drop([" Label"], axis=1)
y_bin = [0 if x=="BENIGN" else 1 for x in y.values]

print("The text labels from the encoding will be passed to classification report so we can interpret our results more easily.\n")
binary_labels = ["BENIGN", "ATTACK"]

print("Binary Label Encodings (in order of digits 0 -> 1): ")
print(binary_labels)

y_multi = y

# Encode attack labels to int and save as array to be used later.
le = LabelEncoder()
y_multi = le.fit_transform(y_multi.values)

# Offset encoding so it corresponds to our original encoding (no benign column, which is now -1)
y_multi = [x-1 for x in y_multi]

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(y_multi))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)

The text labels from the encoding will be passed to classification report so we can interpret our results more easily.

Binary Label Encodings (in order of digits 0 -> 1): 
['BENIGN', 'ATTACK']

Multiclass Label Encodings (in order of digits 0 -> n): 
['BENIGN', 'DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


Load models

In [17]:
# Load a binary model
stage_1 = pickle.load(open("rf_binary_test.pickle", 'rb'))

# Load a multiclass model
stage_2 = keras.models.load_model('LSTM_big.h5')

Stage 1 Prediction (Binary)

In [18]:
y_pred = stage_1.predict(X)

In [19]:
print(classification_report(y_bin, y_pred, digits=6, target_names=binary_labels))

              precision    recall  f1-score   support

      BENIGN   0.999982  1.000000  0.999991     54571
      ATTACK   1.000000  0.999982  0.999991     54571

    accuracy                       0.999991    109142
   macro avg   0.999991  0.999991  0.999991    109142
weighted avg   0.999991  0.999991  0.999991    109142



Housekeeping to prepare the dataset for 2nd stage.

In [20]:
# Create new dataframe out of binary predictions and multiclass labels that can be used to compare them later.
X_agg = X.copy()
X_agg["binary_pred"] = y_pred.copy()
X_agg[" Labels"] = y_multi.copy()

# Slice out the portions that we've identified as benign and replace their label with that of the multiclass benign label for comparison later.
X_rem = X_agg[X_agg["binary_pred"] == 0]
X_rem['binary_pred'] = X_rem['binary_pred'].replace([0], -1)

# Slice out the portions we've labeled as attack for 2nd stage classification.
X_mult = X_agg[X_agg["binary_pred"] == 1]
X_mult = X_mult.drop(["binary_pred"], axis = 1)
y_mult = X_mult[" Labels"].copy()
X_mult = X_mult.drop([" Labels"], axis = 1)

C:\Users\icarus\AppData\Local\Temp\ipykernel_45344\3257394839.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_rem['binary_pred'] = X_rem['binary_pred'].replace([0], -1)


In [21]:
# Some reshaping is needed for the LSTM model to work
scal = StandardScaler()
X_tr = scal.fit_transform(X_mult)
X_tr = np.reshape(X_tr, (X_mult.shape[0], 1, X_mult.shape[1]))

y_pred2 = stage_2.predict(X_tr)
y_pred2 = [np.argmax(x) for x in y_pred2]

1706/1706 [==============================] - 4s 2ms/step


In [24]:
print(classification_report(y_mult, y_pred2, digits=6, target_names=multiclass_labels))

ValueError: Number of classes, 11, does not match size of target_names, 12. Try specifying the labels parameter

Final Calculation of Results

In [25]:
y_pred_final = list(X_rem["binary_pred"].values.copy())
y_pred_final.extend(y_pred2)

y_true_final = list(X_rem[" Labels"].values.copy())
y_true_final.extend(y_mult.values)

print(classification_report(y_true_final, y_pred_final, digits=6, target_names=multiclass_labels))

              precision    recall  f1-score   support

      BENIGN   0.999982  1.000000  0.999991     54571
         DNS   0.416588  0.620641  0.498543      4961
        LDAP   0.277583  0.255594  0.266135      4961
       MSSQL   0.313257  0.536787  0.395632      4961
         NTP   0.612810  0.815763  0.699870      4961
     NetBIOS   0.835032  0.792784  0.813360      4961
        SNMP   0.497365  0.342471  0.405634      4961
        SSDP   0.315071  0.321105  0.318059      4961
         Syn   0.597226  0.997984  0.747264      4961
        TFTP   0.635474  0.209434  0.315039      4961
         UDP   0.458539  0.317678  0.375327      4961
     UDP-lag   0.961538  0.186454  0.312342      4961

    accuracy                       0.745304    109142
   macro avg   0.576706  0.533058  0.512266    109142
weighted avg   0.769104  0.745304  0.733959    109142

